In [ ]:
# Import necessary libraries
import openai
import pandas as pd
import random
import time
import json
import os
from google.colab import drive
from openai import OpenAI


# Set OpenAI API credentials

openai.organization = organization_id
openai.api_key = secret_key
os.environ["OPENAI_API_KEY"] = secret_key
client = OpenAI(organization=organization_id)
OPENAI_API_KEY = "your-api-key-here"
MODEL_NAME = "gpt-3.5-turbo"  # Update with your fine-tuned model name
openai.api_key = OPENAI_API_KEY

# Define file paths
test_data_path = "data/test_dataset.json"
output_csv_path = "data/student_zero_shot_results.csv"

# Load test dataset
with open(test_data_path, "r", encoding="utf-8") as f:
    test_data = json.load(f)

# Convert JSON data to DataFrame
df_test = pd.DataFrame(test_data)

# Select 50 unique (Grade, Topic, Sub-topic) combinations
unique_combinations = df_test[["Grade", "Topic", "Sub Topic"]].drop_duplicates()
selected_combinations = unique_combinations.sample(n=50, random_state=42)

# Convert to DataFrame
math_q_df = pd.DataFrame(selected_combinations, columns=["Grade", "Topic", "Sub Topic", "Challenge Type", "Student Mistake Prompt", "Problem"])

# Save dataset to Google Drive
math_q_df.to_csv("data/selected_50_unique_rows_teacher_model.csv", index=False)

# Function to generate student mistake prompts
def generate_student_mistake(grade, topic, sub_topic):
    prompts = [
        f"A grade {grade} student is struggling with {sub_topic} in {topic}. Generate a common mistake and the problem statement.",
        f"What is a typical error students make in {sub_topic}? Provide a problem where such a mistake happens.",
        f"Create a math problem related to {sub_topic} in {topic} where students usually make errors."
    ]
    start = random.choice(prompts)
    output_format = "Output should be only a valid json with the following format. {\"challenge_type\":\"<what type of challenge they faced>\", \"question\":\"<The problem example they are solving>\", \"mistake\":\"Students message showing the mistake\"}"

    final = start + " " + output_format
    return final

# Function to call OpenAI API and generate problems
def get_math_problem(prompt):
    try:
        response = client.chat.completions.create(
            model=MODEL_NAME,
            messages=[
                {"role": "system", "content": "You are a math tutor generating common student mistakes and problems."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=1000
        )
        output = response.choices[0].message.content
        print(output)
        output = output.replace("```json","")
        output = output.replace("```","")
        output = output.replace("\\","")
        json_o = json.loads(output)
        challenge_type = json_o["challenge_type"]
        question = json_o["question"]
        mistake = json_o["mistake"]
        return [challenge_type, question, mistake]
    except Exception as e:
        print("Error generating response:", e)
        return None

# Generate dataset
math_data = []

for _, row in selected_combinations.iterrows():
    grade = row["Grade"]
    topic = row["Topic"]
    sub_topic = row["Sub Topic"]

    num_samples = 100

    for _ in range(num_samples):

      student_prompt = generate_student_mistake(grade, topic, sub_topic)
      print(student_prompt)
      r = get_math_problem(student_prompt)

      if r:

          challenge_type=r[0]
          question=r[1]
          mistake=r[2]


          math_data.append([grade, topic, sub_topic, challenge_type, question, mistake])

    # Avoid rate limits
    time.sleep(1)

# Convert to DataFrame
math_df = pd.DataFrame(math_data, columns=["Grade", "Topic", "Sub-topic", "Challenge Type", "Student Mistake Prompt", "Problem"])

# Save dataset to Google Drive
math_df.to_csv(output_csv_path, index=False)

print(f" Dataset generation complete. Saved as: {output_csv_path}")


Streaming output truncated to the last 5000 lines.
A grade K5 student is struggling with multi-step word problems in measurement in Measurement. Generate a common mistake and the problem statement. Output should be only a valid json with the following format. {"challenge_type":"<what type of challenge they faced>", "question":"<The problem example they are solving>", "mistake":"Students message showing the mistake"}
{
    "challenge_type": "Difficulty in understanding units of measurement",
    "question": "A farmer has 5 bags of potatoes. Each bag weighs 3 kilograms. If the farmer sells 2 bags of potatoes, how many kilograms of potatoes are left?",
    "mistake": "The student may add 5 and 3 instead of multiplying, and incorrectly state that there are 8 kilograms of potatoes left."
}
Create a math problem related to multi-step word problems in measurement in Measurement where students usually make errors. Output should be only a valid json with the following format. {"challenge_type":